In [36]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [37]:
# import the path
import os
path = os.path.abspath(os.path.join("..", "08_portfolio_management", "portfolio_example.xlsx"))

# read the Excel file
df = pd.read_excel(path, sheet_name=None)

# show the first sheet's data
first_sheet_name = list(df.keys())[0]

# dict to dataframe
df = pd.DataFrame(df[first_sheet_name])

df.head()

,Ticker,Last Closing Price,Somma di Q.ty
0,AAPL,232.560,18.00
1,AMD,168.580,18.00
2,ASML,654.300,6.00
3,BNB,857.400,13.09
4,BR50,21.855,213.00


In [38]:
# add here Last Closing Price data download from yf or tv
# for tv, need to set the correct exchange/market

# import yfinance as yf
# from tvDatafeed import TvDatafeed, Interval
# tv = TvDatafeed()

In [39]:
# rename columns for clarity
df.rename(columns={'Last Closing Price': 'Last_Price', 'Somma di Q.ty': 'Quantity'}, inplace=True)

# calculate portfolio value for each position
df['Position_Value'] = df['Last_Price'] * df['Quantity']

# display the portfolio
print("Portfolio Overview:")
df.head()

Portfolio Overview:


,Ticker,Last_Price,Quantity,Position_Value
0,AAPL,232.560,18.00,4186.080
1,AMD,168.580,18.00,3034.440
2,ASML,654.300,6.00,3925.800
3,BNB,857.400,13.09,11223.366
4,BR50,21.855,213.00,4655.115


In [40]:
print(f"\nTotal Portfolio Value: ${df['Position_Value'].sum():,.2f}")
print(f"Number of positions: {len(df)}")


Total Portfolio Value: $426,292.51
Number of positions: 24


In [41]:
# portfolio pct composition
df['Position_Weight'] = (df['Position_Value'] / df['Position_Value'].sum())*100
df.head()

,Ticker,Last_Price,Quantity,Position_Value,Position_Weight
0,AAPL,232.560,18.00,4186.080,0.981974
1,AMD,168.580,18.00,3034.440,0.711821
2,ASML,654.300,6.00,3925.800,0.920917
3,BNB,857.400,13.09,11223.366,2.632785
4,BR50,21.855,213.00,4655.115,1.092000


In [ ]:
# # download df as csv
# df.to_csv("portfolio_data.csv", index=False)

In [43]:
# top 10 holdings
top_10 = df.nlargest(10, 'Position_Value')
top_10

,Ticker,Last_Price,Quantity,Position_Value,Position_Weight
5,BTC,111968.6300,2.1571,241527.531773,56.657700
16,META,738.7000,60.0000,44322.000000,10.397086
8,ETH,4459.3400,4.3100,19219.755400,4.508584
7,EQQQ,489.6000,39.0000,19094.400000,4.479178
15,MC,513.1000,22.0000,11288.200000,2.647994
3,BNB,857.4000,13.0900,11223.366000,2.632785
12,IWDE,101.5700,96.0000,9750.720000,2.287331
6,DOGE,0.2153,40270.0000,8670.131000,2.033845
22,TSLA,345.9800,23.0000,7957.540000,1.866685
18,NKE,77.9200,79.0000,6155.680000,1.444004
